In [ ]:
# 0. Reference the necessary libraries
from autogen_agentchat.agents import CodingAssistantAgent, ToolUseAssistantAgent,AssistantAgent
from autogen_agentchat.task import Console, TextMentionTermination
from autogen_agentchat.teams import RoundRobinGroupChat
from autogen_core.components.tools import FunctionTool
from autogen_ext.models import OpenAIChatCompletionClient
from AzModelClient import create_azure_openai_client

In [ ]:
# 1.1 Phtyon funciton  code to execute Google Search and return the results in structured way
def google_search(query: str, num_results: int = 2, max_chars: int = 500) -> list:  # type: ignore[type-arg]
    import os
    import time

    import requests
    from bs4 import BeautifulSoup
    from dotenv import load_dotenv

    load_dotenv()

    GOOGLE_API_KEY =  os.getenv("GOOGLE_API_KEY")
    GOOGLE_SEARCH_ENGINE_ID = os.getenv("GOOGLE_SEARCH_ENGINE_ID")

    if not GOOGLE_API_KEY or not GOOGLE_SEARCH_ENGINE_ID:
        raise ValueError("API key or Search Engine ID not found in environment variables")

    url = "https://www.googleapis.com/customsearch/v1"
    params = {"key": GOOGLE_API_KEY, "cx": GOOGLE_SEARCH_ENGINE_ID, "q": query, "num": num_results}

    response = requests.get(url, params=params)  # type: ignore[arg-type]

    if response.status_code != 200:
        print(response.json())
        raise Exception(f"Error in API request: {response.status_code}")

    results = response.json().get("items", [])

    def get_page_content(url: str) -> str:
        try:
            response = requests.get(url, timeout=10)
            soup = BeautifulSoup(response.content, "html.parser")
            text = soup.get_text(separator=" ", strip=True)
            words = text.split()
            content = ""
            for word in words:
                if len(content) + len(word) + 1 > max_chars:
                    break
                content += " " + word
            return content.strip()
        except Exception as e:
            print(f"Error fetching {url}: {str(e)}")
            return ""

    enriched_results = []
    for item in results:
        body = get_page_content(item["link"])
        enriched_results.append(
            {"title": item["title"], "link": item["link"], "snippet": item["snippet"], "body": body}
        )
        time.sleep(1)  # Be respectful to the servers

    return enriched_results

In [ ]:
# 1.2 Phtyon funciton  code to execue ARXIV Search and return the results in structured way
def arxiv_search(query: str, max_results: int = 2) -> list:  # type: ignore[type-arg]
    """
    Search Arxiv for papers and return the results including abstracts.
    """
    import arxiv

    client = arxiv.Client()
    search = arxiv.Search(query=query, max_results=max_results, sort_by=arxiv.SortCriterion.Relevance)

    results = []
    for paper in client.results(search):
        results.append(
            {
                "title": paper.title,
                "authors": [author.name for author in paper.authors],
                "published": paper.published.strftime("%Y-%m-%d"),
                "abstract": paper.summary,
                "pdf_url": paper.pdf_url,
            }
        )
    return results

In [ ]:
# 1.3 Tool Definitions for the Agents

#Search in google for
# {"title": item["title"], "link": item["link"], "snippet": item["snippet"], "body": body}
google_search_tool = FunctionTool(
    google_search, description="Search Google for information, returns results with a snippet and body content"
)

#Search in arxiv for
#"title";  "authors"; "published"; "abstract";summary,"pdf_url": paper.pdf_url,
arxiv_search_tool = FunctionTool(
    arxiv_search, description="Search Arxiv for papers related to a given topic, including abstracts"
)


In [ ]:
# 2. Agent Definitions
# 2.1 Google Search Agent
google_search_agent = AssistantAgent(
    name="Google_Search_Agent",
    tools=[google_search_tool],
    model_client=create_azure_openai_client(),
    description="An agent that can search Google for information, returns results with a snippet and body content",
    system_message="You are a helpful AI assistant. Solve tasks using your tools.",
)

# 2.2 Arxiv Search Agent
arxiv_search_agent = AssistantAgent(
    name="Arxiv_Search_Agent",
    tools=[arxiv_search_tool],
    model_client=create_azure_openai_client(),
    description="An agent that can search Arxiv for papers related to a given topic, including abstracts",
    system_message="""You are a helpful AI assistant. Solve tasks using your tools. Specifically, 
    you can take into consideration the user's request and craft a search query that is most likely 
    to return relevant academi papers.""",
)

# 2.3 Report Agent
report_agent = AssistantAgent(
    name="Report_Agent",
    model_client=create_azure_openai_client(),
    description="Generate a report based on a given topic",
    system_message="""You are a helpful assistant. 
    Your task is to synthesize data extracted into a high quality literature review including CORRECT references. 
    You MUST write a final report that is formatted as a literature review with CORRECT references.  
    Your response should end with the word 'TERMINATE'""",
)

In [ ]:
#3. TEam or agent team definitions
# 3.1 Termination confitions for the Group chat
termination = TextMentionTermination("TERMINATE")

# 3.2 Team definition, using ROund Robin Group Chat and the order of the agents
team = RoundRobinGroupChat(
    participants=[arxiv_search_agent, google_search_agent, report_agent], termination_condition=termination
)

In [ ]:
# 4. Run the team
stream =  team.run_stream(task="Write a literature review on no code tools for building multi agent ai systems")
#4.2 print the stream outcomes.
await Console(stream)